# Intereacting with the shell

## `sh` module

The [`sh` module](https://amoffat.github.io/sh/) is very convenient to interact with the shell.  Note that `sh` is not part of Python's standard library, if you prefer not to use extra modules, use the `subprocess` module in the standard library.

In [1]:
import sh

Any shell command can be executed by calling it as a function on the `sh` module, passing command line arguments as arguments.

In [2]:
sh.ls('-l')

total 48
-rw-r--r-- 1 gjb gjb    86 Nov  7 17:56 README.md
-rw-r--r-- 1 gjb gjb 20943 Nov 12 13:23 shell_interaction.ipynb
-rwxrw-r-- 1 gjb gjb   858 Aug 23 16:56 subprocess_environment.py
-rwxrw-r-- 1 gjb gjb  2439 Aug 20 08:56 sys_info.py
-rw-r--r-- 1 gjb gjb  8616 Nov  8 11:54 system_information.ipynb

The output can be used by assigning the command to a variable, and using the result's `stdout` attribute.  Note that the latter is a sequence of bytes, so it has to be decoded into a UTF-8 string for further processing.

In [3]:
cmd = sh.ls('-l', '-a', _encoding='UTF-8')

In [4]:
lines = cmd.stdout.decode(encoding='utf8').split('\n')

In [5]:
len(lines[1:-1])

8

In [6]:
lines[1:-1]

['drwxrwxr-x  3 gjb gjb  4096 Nov 12 13:23 .',
 'drwxrwxr-x 79 gjb gjb  4096 Oct 12 15:57 ..',
 'drwxr-xr-x  2 gjb gjb  4096 Nov 11 09:20 .ipynb_checkpoints',
 '-rw-r--r--  1 gjb gjb    86 Nov  7 17:56 README.md',
 '-rw-r--r--  1 gjb gjb 20943 Nov 12 13:23 shell_interaction.ipynb',
 '-rwxrw-r--  1 gjb gjb   858 Aug 23 16:56 subprocess_environment.py',
 '-rwxrw-r--  1 gjb gjb  2439 Aug 20 08:56 sys_info.py',
 '-rw-r--r--  1 gjb gjb  8616 Nov  8 11:54 system_information.ipynb']

In [7]:
sh.mkdir('tmp', '-p')

In [8]:
sh.ls()

README.md		 subprocess_environment.py  system_information.ipynb
shell_interaction.ipynb  sys_info.py		    tmp

### Exit codes

When a shell command fails, an exception is thrown which contains the full command as it was run, the exit code, the standard output and error.

In [9]:
try:
    sh.ls('bla.txt')
except Exception as error:
    err_msg = error.stderr.decode(encoding='utf8').rstrip()
    print(f'command "{error.full_cmd}" exited with exit code {error.exit_code} and message "{err_msg}"')

command "/bin/ls bla.txt" exited with exit code 2 and message "/bin/ls: cannot access 'bla.txt': No such file or directory"


### I/O redirection

Redirecting output can be done using the `_out` optional argument.

In [10]:
with open('tmp/date_file.txt', 'w') as file:
    for i in range(10):
        print(f'{i} ', end='', file=file)
        sh.date(_out=file)
        sh.sleep('1')

In [11]:
sh.cat('tmp/date_file.txt')

0 Mon Nov 12 13:24:26 CET 2018
1 Mon Nov 12 13:24:27 CET 2018
2 Mon Nov 12 13:24:28 CET 2018
3 Mon Nov 12 13:24:29 CET 2018
4 Mon Nov 12 13:24:30 CET 2018
5 Mon Nov 12 13:24:31 CET 2018
6 Mon Nov 12 13:24:32 CET 2018
7 Mon Nov 12 13:24:33 CET 2018
8 Mon Nov 12 13:24:34 CET 2018
9 Mon Nov 12 13:24:35 CET 2018

Input redirection works similarly using the optional `_in` argument.

In [12]:
with open('tmp/date_file.txt', 'r') as file:
    print(sh.wc('-l', _in=file))

10



### Piping

The output of a command can be used as the input for another command.

Pipe the output of `ls` into `grep` to select only the files with names that end with `.py`.

In [13]:
sh.grep(sh.ls('-l'), r'\.py$')

-rwxrw-r-- 1 gjb gjb   858 Aug 23 16:56 subprocess_environment.py
-rwxrw-r-- 1 gjb gjb  2439 Aug 20 08:56 sys_info.py

Pipe the output of `cut` into `sort`.

In [14]:
sh.sort(sh.cut('-d', ' ', '-f', '5', 'tmp/date_file.txt'), '-r')

13:24:35
13:24:34
13:24:33
13:24:32
13:24:31
13:24:30
13:24:29
13:24:28
13:24:27
13:24:26

In [15]:
sh.rm('-r', 'tmp')

### Backgrounding & time out

Long running processes can be placed in the background.

In [16]:
process = sh.sleep(10, _bg=True)

In [17]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [18]:
process.wait()

In [19]:
print(process.exit_code)

0


A time out can be specified for a command, and on time out, the resulting exit code will be the number of the signal (SIGKILL by default).

In [20]:
try:
    process = sh.sleep(10, _bg=True, _timeout=3)
except TimeoutError as error:
    print(error)

## `subprocess` module

If you prefer to use standard library modules only, `subprocess` is a good choice.

In [21]:
import subprocess

This module has a high-level function `run` that can be used for almost any processing.  The API is still being improved in subsequent releases of Python.

In [22]:
process = subprocess.run(['ls', '-l'], stdout=subprocess.PIPE, encoding='utf8')

In [23]:
process.stdout.split('\n')

['total 48',
 '-rw-r--r-- 1 gjb gjb    86 Nov  7 17:56 README.md',
 '-rw-r--r-- 1 gjb gjb 20943 Nov 12 13:23 shell_interaction.ipynb',
 '-rwxrw-r-- 1 gjb gjb   858 Aug 23 16:56 subprocess_environment.py',
 '-rwxrw-r-- 1 gjb gjb  2439 Aug 20 08:56 sys_info.py',
 '-rw-r--r-- 1 gjb gjb  8616 Nov  8 11:54 system_information.ipynb',
 '']

Note that if you don't specify the `stdout` arugment, the output of the command will not be captured.  Python 3.7 makes this easier by adding a `capture_output` argument.

### Exit codes

The `run` function returns a `CompletedProcess` object that has an attribute for the exit code returned by the process.

In [24]:
process = subprocess.run(['mkdir', '-p', 'tmp'])

In [25]:
process.returncode

0

### I/O redirection

Output of a running command can be redirected to a file.

In [26]:
with open('tmp/data.txt', 'w') as file:
    for i in range(10):
        subprocess.run(['echo', '-n', str(i) + ' '], stdout=file)
        subprocess.run(['date'], stdout=file)
        subprocess.run(['sleep', '1'])

Exception in thread background thread for pid 23198:
Traceback (most recent call last):
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 1540, in wrap
    fn(*args, **kwargs)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 2459, in background_thread
    handle_exit_code(exit_code)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 2157, in fn
    return self.command.handle_command_exit_code(exit_code)
  File "/home/gjb/miniconda3/envs/py36/lib/python3.6/site-packages/sh.py", line 815, in handle_command_exit_code
    raise exc
sh.SignalException_SIGKILL: 

  RAN: /bin/sleep 10

  STDOUT:


  STDERR:




Note that mixed I/O from the `print` function and `run` doesn't work as expected.

In [27]:
print(subprocess.run(['cat', 'tmp/data.txt'], stdout=subprocess.PIPE,
                     encoding='utf8').stdout)

0 Mon Nov 12 13:24:48 CET 2018
1 Mon Nov 12 13:24:49 CET 2018
2 Mon Nov 12 13:24:50 CET 2018
3 Mon Nov 12 13:24:51 CET 2018
4 Mon Nov 12 13:24:52 CET 2018
5 Mon Nov 12 13:24:53 CET 2018
6 Mon Nov 12 13:24:54 CET 2018
7 Mon Nov 12 13:24:55 CET 2018
8 Mon Nov 12 13:24:56 CET 2018
9 Mon Nov 12 13:24:57 CET 2018



Input redirection is similar.

In [28]:
with open('tmp/data.txt', 'r') as file:
    process = subprocess.run(['wc', '-l'], stdin=file, stdout=subprocess.PIPE,
                             encoding='utf8')
    print(process.stdout)

10



### Piping

Piping can also be done using `subprocess`.  It is less user friendly than using the `sh` module, but it allows more control.  You will have to resort to the low-level `Popen` function.

In [29]:
p1 = subprocess.Popen(['ls', '-l'], stdout=subprocess.PIPE)
p2 = subprocess.Popen(['grep', r'\.py$'], stdin=p1.stdout, stdout=subprocess.PIPE, encoding='utf8')
p1.stdout.close()
output, _ = p2.communicate()
print(output)

-rwxrw-r-- 1 gjb gjb   858 Aug 23 16:56 subprocess_environment.py
-rwxrw-r-- 1 gjb gjb  2439 Aug 20 08:56 sys_info.py



In [30]:
p1 = subprocess.Popen(['cut', '-d', ' ', '-f', '5', 'tmp/data.txt'], stdout=subprocess.PIPE)
p2 = subprocess.Popen(['sort', '-r'], stdin=p1.stdout, stdout=subprocess.PIPE, encoding='utf8')
p1.stdout.close()
output, _ = p2.communicate()
print(output)

13:24:57
13:24:56
13:24:55
13:24:54
13:24:53
13:24:52
13:24:51
13:24:50
13:24:49
13:24:48



In [32]:
_ = subprocess.run(['rm', '-r', 'tmp'])